In [19]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import random
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.inspection import DecisionBoundaryDisplay

In [20]:
con = sqlite3.connect(r"../laliga.sqlite")
# Change date format (2029 -> 1929)
df = pd.read_sql_query("SELECT * FROM Matches", con)

# Data formatting:
# We split the "date" column into the corresponding columns, and specify the correct formatting.
# This way we avoid the year 29 or higher to be mistaken by 2029 (error with to_datetime function).
df[["month", "day", "year"]] = df["date"].str.split("/", 2, expand = True)
df["year"] = df["year"].apply(lambda x: f"19{x}" if int(x) >= 23 else f"20{x}")
df['date'] = pd.to_datetime(df[['day', 'month', 'year']]).dt.strftime('%d/%m/%Y') 

# We also change the formatting of the time to be in 24h format
df['time'] = df['time'].apply(lambda x: datetime.strftime(datetime.strptime(x, "%I:%M %p"), "%H:%M") if type(x) == str else x)

df[["home_goals", "away_goals"]] = df["score"].str.split(":", expand=True).astype(float)
df2=df.dropna(subset = "score")
df2["goal diff"] = (df2["home_goals"]  - df2["away_goals"])
df2["results"] = np.where(df2["goal diff"] > 0, "1", np.where(df2["goal diff"] < 0, "2", "X"))
df2['home_win'] = df['home_goals'] > df['away_goals']
df2.head()

,season,division,matchday,date,time,home_team,away_team,score,month,day,year,home_goals,away_goals,goal diff,results,home_win
0,1928-1929,1,1,10/02/1929,None,Arenas Club,Athletic Madrid,2:3,2,10,1929,2.0,3.0,-1.0,2,False
1,1928-1929,1,1,10/02/1929,None,Espanyol,Real Unión,3:2,2,10,1929,3.0,2.0,1.0,1,True
2,1928-1929,1,1,10/02/1929,None,Real Madrid,Catalunya,5:0,2,10,1929,5.0,0.0,5.0,1,True
3,1928-1929,1,1,10/02/1929,None,Donostia,Athletic,1:1,2,10,1929,1.0,1.0,0.0,X,False
4,1928-1929,1,1,12/02/1929,None,Racing,Barcelona,0:2,2,12,1929,0.0,2.0,-2.0,2,False


In [21]:
def parse_seasons(value):
    if value == "all":
        return "all"
    seasons = []
    for chunk in value.split(","):
        if ":" in chunk:
            try:
                start, end = map(int, chunk.split(":"))
                assert start < end
            except Exception:
                print("error")
            for i in range(start, end):
                seasons.append(f"{i}-{i+1}")
        else:
            try:
                start, end = map(int, chunk.split("-"))
                assert start == end - 1
            except Exception:
                print("error")
            seasons.append(chunk)
    return seasons

In [27]:
training_seasons = parse_seasons("2006:2009")

In [28]:
x_train = df2[df2["season"].isin(training_seasons)]
x_train

,season,division,matchday,date,time,home_team,away_team,score,month,day,year,home_goals,away_goals,goal diff,results,home_win
20358,2006-2007,1,1,26/08/2006,22:00,Valencia,Real Betis,2:1,8,26,2006,2.0,1.0,1.0,1,True
20359,2006-2007,1,1,27/08/2006,19:00,Espanyol,Gimnàstic,0:1,8,27,2006,0.0,1.0,-1.0,2,False
20360,2006-2007,1,1,27/08/2006,19:00,Dep. La Coruña,Real Zaragoza,3:2,8,27,2006,3.0,2.0,1.0,1,True
20361,2006-2007,1,1,27/08/2006,19:00,CA Osasuna,Getafe,0:2,8,27,2006,0.0,2.0,-2.0,2,False
20362,2006-2007,1,1,27/08/2006,19:00,Real Madrid,Villarreal,0:0,8,27,2006,0.0,0.0,0.0,X,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42629,2008-2009,2,42,20/06/2009,18:30,Elche CF,Real Sociedad,1:2,6,20,2009,1.0,2.0,-1.0,2,False
42630,2008-2009,2,42,20/06/2009,18:30,Alicante CF,Sevilla Atl.,3:3,6,20,2009,3.0,3.0,0.0,X,False
42631,2008-2009,2,42,20/06/2009,20:00,Córdoba CF,Real Murcia,2:1,6,20,2009,2.0,1.0,1.0,1,True
42632,2008-2009,2,42,20/06/2009,20:15,SD Huesca,Girona,3:1,6,20,2009,3.0,1.0,2.0,1,True


In [ ]:
def direct_confrontations(Team1, Team2)
    df3 = df2.loc[((df2["home_team"] == Team2) & (df2["away_team"] == Team1)) | ((df2["home_team"] == Team1) & (df2["away_team"] == Team2))]
    df3["Winner"] = np.where(df3["goal diff"] > 0, df3.home_team.values, np.where(df3["goal diff"] < 0, df3.away_team.values, "tie"))
    direct_confrontation_count = df3["Winner"].value_counts()

    return direct_confrontation_count

In [ ]:
counts = direct_confrontations()